pip install gensim==3.8.3

!pip install bert-extractive-summarizer

pip install keybert
pip install gensim
pip install transformers

## Article Selection

In [1]:
import pandas as pd

articles = pd.read_csv('train.csv',on_bad_lines='skip', header = None)
articles.fillna(' ', inplace=True)
articles.head()

,0,1,2
0,Sport;21-Jähriger fällt wohl bis Saisonende au...,wie eine Magnetresonanz-Untersuchung am Donne...,muss aber nicht operiert werden. Dennoch steh...
1,Sport;Traditionsklub setzt sich gegen den FC U...,Utrecht war Fünfter geworden.,
2,International;Militär setzt Offensive an Grenz...,teilte das Militär in Islamabad mit. Es handl...,
3,International;Kämpfe halten trotz vereinbarter...,wie Präsidialamtssprecher Andrej Lyssenko am ...,hieß es. Im Rahmen des in Minsk (Weißrussland...
4,Kultur;Der in Graz ansässige Schriftsteller g...,in Graz ansässige Autor Helmut Schranz ist am...,die heuer im April im Ritter-Verlag (Klagenfu...


In [2]:
articles['new'] = articles.apply(''.join, axis=1)
articles_new = articles[['new']]
articles_new[['Topic','Article']] = articles_new.new.str.split(";",n = 1, expand = True)
articles_new.head()

/var/folders/vg/7d4f1ys12s17fxmrbfx48h4h0000gn/T/ipykernel_42327/1983126588.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  articles_new[['Topic','Article']] = articles_new.new.str.split(";",n = 1, expand = True)
/var/folders/vg/7d4f1ys12s17fxmrbfx48h4h0000gn/T/ipykernel_42327/1983126588.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  articles_new[['Topic','Article']] = articles_new.new.str.split(";",n = 1, expand = True)


,new,Topic,Article
0,Sport;21-Jähriger fällt wohl bis Saisonende au...,Sport,21-Jähriger fällt wohl bis Saisonende aus. Wie...
1,Sport;Traditionsklub setzt sich gegen den FC U...,Sport,Traditionsklub setzt sich gegen den FC Utrecht...
2,International;Militär setzt Offensive an Grenz...,International,Militär setzt Offensive an Grenze zu Afghanist...
3,International;Kämpfe halten trotz vereinbarter...,International,Kämpfe halten trotz vereinbarter Waffenruhe an...
4,Kultur;Der in Graz ansässige Schriftsteller g...,Kultur,Der in Graz ansässige Schriftsteller gehörte ...


In [3]:
final_df = articles_new[['Topic','Article']]
final_df.head()

,Topic,Article
0,Sport,21-Jähriger fällt wohl bis Saisonende aus. Wie...
1,Sport,Traditionsklub setzt sich gegen den FC Utrecht...
2,International,Militär setzt Offensive an Grenze zu Afghanist...
3,International,Kämpfe halten trotz vereinbarter Waffenruhe an...
4,Kultur,Der in Graz ansässige Schriftsteller gehörte ...


In [85]:
import numpy as np

rand = final_df[final_df.Topic == 'Sport']

rows = np.random.choice(rand.index.values, 1)
sampled_df = final_df.loc[rows]
sentence = sampled_df['Article'].values[0]

In [86]:
sentence

'Spielberg – Nico Rosberg hat am Sonntag auch die zweite Auflage des Formel-1-Grand-Prix  von Österreich nach dem Comeback im Vorjahr gewonnen. Der 29-jährige Deutsche  triumphierte vor 55.000 Zuschauern in Spielberg vor seinem Mercedes-Stallrivalen  Lewis Hamilton und Felipe Massa im Williams. Ferrari-Star Sebastian Vettel wurde  nach einem verpatzten Boxenstopp Vierter. Rosberg dagegen bejubelte den elften Grand-Prix-Sieg seiner Karriere den  dritten in dieser Saison. In der WM rückte er dem führenden Titelverteidiger  Hamilton wieder bis auf zehn Punkte nahe. Das nächste Rennen geht in zwei Wochen  in Silverstone über die Bühne. '

In [87]:
sen = str(sentence).split('. ')
sen

['Spielberg – Nico Rosberg hat am Sonntag auch die zweite Auflage des Formel-1-Grand-Prix  von Österreich nach dem Comeback im Vorjahr gewonnen',
 'Der 29-jährige Deutsche  triumphierte vor 55.000 Zuschauern in Spielberg vor seinem Mercedes-Stallrivalen  Lewis Hamilton und Felipe Massa im Williams',
 'Ferrari-Star Sebastian Vettel wurde  nach einem verpatzten Boxenstopp Vierter',
 'Rosberg dagegen bejubelte den elften Grand-Prix-Sieg seiner Karriere den  dritten in dieser Saison',
 'In der WM rückte er dem führenden Titelverteidiger  Hamilton wieder bis auf zehn Punkte nahe',
 'Das nächste Rennen geht in zwei Wochen  in Silverstone über die Bühne',
 '']

# BERT Translator

In [88]:
# BERT Translator

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/bert2bert_L-24_wmt_de_en", pad_token="<pad>", eos_token="</s>", bos_token="<s>")
model = AutoModelForSeq2SeqLM.from_pretrained("google/bert2bert_L-24_wmt_de_en")



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [89]:
output = []
for i in sen:
    if len(i)>1:
        print(i)
        input_ids = tokenizer(i, return_tensors="pt", add_special_tokens=False).input_ids
        output_ids = model.generate(input_ids)[0]
        translated_text = tokenizer.decode(output_ids, skip_special_tokens=True)
        output.append(translated_text)

Spielberg – Nico Rosberg hat am Sonntag auch die zweite Auflage des Formel-1-Grand-Prix  von Österreich nach dem Comeback im Vorjahr gewonnen


/Users/sumitha/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/transformers/generation/utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 128 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Der 29-jährige Deutsche  triumphierte vor 55.000 Zuschauern in Spielberg vor seinem Mercedes-Stallrivalen  Lewis Hamilton und Felipe Massa im Williams
Ferrari-Star Sebastian Vettel wurde  nach einem verpatzten Boxenstopp Vierter
Rosberg dagegen bejubelte den elften Grand-Prix-Sieg seiner Karriere den  dritten in dieser Saison
In der WM rückte er dem führenden Titelverteidiger  Hamilton wieder bis auf zehn Punkte nahe
Das nächste Rennen geht in zwei Wochen  in Silverstone über die Bühne


# Summarizer using Tokenization

In [90]:
import nltk
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize 

In [91]:
value = ' '.join(output)

In [92]:
value

'spielberg – nico rosberg also won the second edition of the formel - 1 - grand - prix of Austria on Sunday after the comeback of the previous year. the 29-year-old German triumphed in front of 55.000 spectators in spielberg before his mercedes - stall rival lewis hamilton and felipe massa in the williams. Ferrari - star sebastian vettel took fourth place after an unsuccessful boxing stop. rosberg, on the other hand, cheered the eleventh Grand-Prix - winner of his career the third in this season. in the wm he rucked again the leading title defender hamilton close to ten points. the best run goes in two weeks in silverstone over the bridge.'

In [93]:
tokens = word_tokenize(value) 

In [94]:
nltk.download("stopwords")
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sumitha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [95]:
punctuation = punctuation + '\n'
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~\n'

In [96]:
word_frequencies = {}
for word in tokens:    
    if word.lower() not in stop_words:
        if word.lower() not in punctuation:
            if word not in word_frequencies.keys():
                word_frequencies[word] = 1
            else:
                word_frequencies[word] += 1

In [97]:
max_frequency = max(word_frequencies.values())
print(max_frequency)

2


In [98]:
for word in word_frequencies.keys():
    word_frequencies[word] = word_frequencies[word]/max_frequency

In [99]:
sent_token = sent_tokenize(value)
sent_token

['spielberg – nico rosberg also won the second edition of the formel - 1 - grand - prix of Austria on Sunday after the comeback of the previous year.',
 'the 29-year-old German triumphed in front of 55.000 spectators in spielberg before his mercedes - stall rival lewis hamilton and felipe massa in the williams.',
 'Ferrari - star sebastian vettel took fourth place after an unsuccessful boxing stop.',
 'rosberg, on the other hand, cheered the eleventh Grand-Prix - winner of his career the third in this season.',
 'in the wm he rucked again the leading title defender hamilton close to ten points.',
 'the best run goes in two weeks in silverstone over the bridge.']

In [100]:
sentence_scores = {}
for sent in sent_token:
    sentence = sent.split(" ")
    for word in sentence:        
        if word.lower() in word_frequencies.keys():
            if sent not in sentence_scores.keys():
                sentence_scores[sent] = word_frequencies[word.lower()]
            else:
                sentence_scores[sent] += word_frequencies[word.lower()]

In [101]:
sentence_scores

{'spielberg – nico rosberg also won the second edition of the formel - 1 - grand - prix of Austria on Sunday after the comeback of the previous year.': 7.5,
 'the 29-year-old German triumphed in front of 55.000 spectators in spielberg before his mercedes - stall rival lewis hamilton and felipe massa in the williams.': 7.5,
 'Ferrari - star sebastian vettel took fourth place after an unsuccessful boxing stop.': 4.0,
 'rosberg, on the other hand, cheered the eleventh Grand-Prix - winner of his career the third in this season.': 2.5,
 'in the wm he rucked again the leading title defender hamilton close to ten points.': 4.5,
 'the best run goes in two weeks in silverstone over the bridge.': 3.0}

In [102]:
from heapq import nlargest

In [103]:
select_length = int(len(sent_token)*0.3)
select_length

1

In [104]:
summary = nlargest(select_length, sentence_scores, key = sentence_scores.get)
summary

['spielberg – nico rosberg also won the second edition of the formel - 1 - grand - prix of Austria on Sunday after the comeback of the previous year.']

In [105]:
summary_sentence = ''.join(summary)
summary_sentence

'spielberg – nico rosberg also won the second edition of the formel - 1 - grand - prix of Austria on Sunday after the comeback of the previous year.'

## BERT Summarizer

In [106]:
from keybert import KeyBERT

In [107]:
from summarizer import Summarizer
model = Summarizer()

from tqdm import tqdm
bert_predicted_summary = []

x = model(str(value))
bert_predicted_summary.append(x)


Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [108]:
bert_predicted_summary

['spielberg – nico rosberg also won the second edition of the formel - 1 - grand - prix of Austria on Sunday after the comeback of the previous year. the 29-year-old German triumphed in front of 55.000 spectators in spielberg before his mercedes - stall rival lewis hamilton and felipe massa in the williams.']

In [109]:
summary_text = bert_predicted_summary[0].split('.')
summary_text

['spielberg – nico rosberg also won the second edition of the formel - 1 - grand - prix of Austria on Sunday after the comeback of the previous year',
 ' the 29-year-old German triumphed in front of 55',
 '000 spectators in spielberg before his mercedes - stall rival lewis hamilton and felipe massa in the williams',
 '']

# Keyword Extractor

In [110]:
kw_model = KeyBERT(model='all-mpnet-base-v2')

keywords = kw_model.extract_keywords(bert_predicted_summary, 

                                     keyphrase_ngram_range=(1, 1), 

                                     stop_words='english', 

                                     highlight=False,

                                     top_n=10)

keywords_list= list(dict(keywords).keys())

print(keywords_list)

['rosberg', 'prix', 'formel', 'mercedes', 'nico', 'won', 'german', 'spielberg', 'austria', 'triumphed']


In [111]:
keywords

[('rosberg', 0.5165),
 ('prix', 0.3789),
 ('formel', 0.364),
 ('mercedes', 0.3413),
 ('nico', 0.2904),
 ('won', 0.2524),
 ('german', 0.2277),
 ('spielberg', 0.2264),
 ('austria', 0.2243),
 ('triumphed', 0.2068)]

# Distractor Generator

In [112]:
glove_file = 'glove.6B.300d.txt'
tmp_file = 'word2vec-glove.6B.300d.txt'

In [113]:
import gensim
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors

In [114]:
from gensim.scripts.glove2word2vec import glove2word2vec
glove2word2vec(glove_file, tmp_file)


(400000, 300)

In [115]:
model = KeyedVectors.load_word2vec_format(tmp_file)
def generate_distractors(answer, count):
    answer = str.lower(answer) 
    closestWords = model.most_similar(positive=[answer], topn=count+1)
    distractors = list(map(lambda x: x[0], closestWords))
    return distractors


In [116]:
import re
import random
def generate_questions(summary):
    for i in summary:
        for j in keywords_list[3:]:
            if j in i.lower():
                op = []
                compiled = re.compile(re.escape(j), re.IGNORECASE)
                print(compiled.sub('______', i))
                options = generate_distractors(j,4)
                op.append(j)
                for i in options:
                    if i not in j and j not in i:
                        op.append(i)
                random.shuffle(op)
                print(op)
                print('')

# Final Multi-Choice Questions

In [117]:
generate_questions(summary_text)

spielberg – ______ rosberg also won the second edition of the formel - 1 - grand - prix of Austria on Sunday after the comeback of the previous year
['hülkenberg', 'rosberg', 'nico', 'hulkenberg', 'heidfeld', 'douga']

 the 29-year-old ______ triumphed in front of 55
['austrian', 'berlin', 'polish', 'german']

000 spectators in spielberg before his ______ - stall rival lewis hamilton and felipe massa in the williams
['daimler', 'bmw', 'lexus', 'audi', 'mercedes', 'benz']

